In [ ]:
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np
import cv2
import requests
import time

In [ ]:
def notifyFile(filename):
    file = {'imageFile':open(filename,'rb')}
    payload = {'message': 'ไม่สวมใส่หน้ากาก'}
    url = 'https://notify-api.line.me/api/notify'
    token = '1ErmEHi2aDBQiswdjRvbsRTr4E5NE9wVpVl5u7FwkD4' #EDIT
    headers = {'Authorization':'Bearer ' + token}

    return requests.post(url, headers=headers , data = payload, files=file)
    #return _lineNotify(payload,file)

In [ ]:
#def _lineNotify(payload,file=None):
    #import requests
    #url = 'https://notify-api.line.me/api/notify'
    #token = '1ErmEHi2aDBQiswdjRvbsRTr4E5NE9wVpVl5u7FwkD4' #EDIT
    #headers = {'Authorization':'Bearer ' + token}
    #return requests.post(url, headers=headers , data = payload, files=file)

In [ ]:
size = (224, 224)
data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

# Disable scientific notation for clarity
np.set_printoptions(suppress=True)

# Load the model
model = tensorflow.keras.models.load_model('keras_model.h5')
# Window name in which image is displayed 
window_name = 'Detect Mask'

# font 
font = cv2.FONT_HERSHEY_SIMPLEX 
  
# org 
org = (50, 50) 
  
# fontScale 
fontScale = 1
   
# Blue color in BGR 
text_color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2

cooldown = 20
last_send = time.time() 

In [ ]:
cap = cv2.VideoCapture(0)
frame_count = 0
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    #if cv2.waitKey(1) & 0xFF == ord('t'):
    cv2.imwrite('./image_frame/'+'frame_{}.jpg'.format(frame_count), frame) 
    files = {'src_img':open('./image_frame/'+'frame_{}.jpg'.format(frame_count), 'rb')}
    url = "https://api.aiforthai.in.th/person/human_detect/"
    data = {'json_export':'true','img_export':'true'}
    headers = {
        'Apikey': "E6PjCdHVmnlyoMJx4xCWQpa2DpT4N3BV" #EDIT
    }
    response = requests.post(url, files=files, headers=headers, data=data)
    print(response.json())
    json = response.json()
    aiforthai = frame.copy()
    persons = json['json_data']
    person_count = 0
    for p in persons:
        print(person_count, p)
        if p['object'] == 'person':
            crop_img = aiforthai[p['y0']:p['y1'], p['x0']:p['x1']]
            cv2.imwrite('person_{}.jpg'.format(person_count), crop_img)
            color = cv2.cvtColor(crop_img, cv2.COLOR_BGR2RGB)
            resized = cv2.resize(color, size, interpolation = cv2.INTER_AREA)
            normalized_image_array = (resized.astype(np.float32) / 127.0) - 1
            data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
            data[0] = normalized_image_array
            prediction = model.predict(data)
            classes = np.argmax(prediction, axis=-1)[0]
            print(classes)
            frame = cv2.rectangle(frame, (p['x0'],p['y0']) , (p['x1'],p['y1']), text_color, thickness) 
            if classes== 0:
                frame = cv2.putText(frame, 'Mask', (p['x0'],p['y0']), font,  
                                    fontScale, text_color, thickness)
                #cv2.imwrite('./image_alert/'+'output_{}.jpg'.format(frame_count), frame)
            elif classes== 1:
                frame = cv2.putText(frame, 'No Mask', (p['x0'],p['y0']), font, fontScale, text_color, thickness)
                img_name = './image_alert/'+'output_{}.jpg'.format(frame_count)
                cv2.imwrite(img_name, frame)
                #print("{} written!".format(img_name))
                if (time.time() - last_send > cooldown):
                    last_send = time.time()
                    notifyFile(img_name)
                    
            else: 
                frame = cv2.putText(frame, '????', (p['x0'],p['y0']), font,  
                                        fontScale, text_color, thickness)
                # Display the resulting frame
                
                person_count = person_count + 1
    frame_count = frame_count + 1
        
        
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    
# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()